In [2]:
# Update sklearn to prevent version mismatches
# !pip install sklearn --upgrade
# install joblib to save model. 
# !pip install joblib

In [14]:
import pandas as pd
import tensorflow
import numpy as np

# Load the data

In [16]:
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test
%store -r X_train_scaled
%store -r X_test_scaled 
%store -r X_train_stand
%store -r X_test_stand

# Train the Model

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
model = LogisticRegression(max_iter = 1000)

In [21]:
model.fit(X_train, y_train)
print(f"The cross-validated accuracy for unprocessed training set:\n{round(cross_val_score(model,X_train, y_train,scoring='accuracy', cv=5).mean(),2)}")
# failed to converge without data's being scaled or standardized: accuracy: 66%

/Users/jiaping/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/jiaping/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

The cross-validated accuracy for unprocessed training set:
0.66


/Users/jiaping/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:
print(f"The cross-validated accuracy for normalized training set: \n{round(cross_val_score(model,X_train_scaled, y_train,scoring='accuracy', cv=5).mean()*100,2)}%")
print(f"The cross-validated accuracy for standardized training set:\n{round(cross_val_score(model,X_train_stand, y_train,scoring='accuracy', cv=5).mean()*100,2)}%")

The cross-validated accuracy for normalized training set: 
84.89%
The cross-validated accuracy for standardized training set:
88.67%


Standardization produces higher cross-validated accuracy than normalization, so standardized data will be used for hyperparameter tuning.

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [23]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import classification_report

param_grid = {'C': [0.00001, 0.0001,0.001,0.01,0.1, 1, 10, 100, 1000], 
              'max_iter': [1000, 5000, 10000],
              'penalty': ["l1", "l2"],
              'solver': ['liblinear', 'lbfgs']}

# Because the "liblinear" solver does not improve the model performance and also makes the search much slower compared to "lbfgs" solver
# grid search about solver and penalty is deleted (lbfgs solver only deal with l2 penalty) for the gridsearch on standardized data
param_grid2 = {'C': [0.00001, 0.0001,0.001,0.01,0.1, 1, 10, 100, 1000], 
              'max_iter': [1000, 5000, 10000]}
model = LogisticRegression()


### 1. Normalized data

In [24]:
grid1 = GridSearchCV(model, param_grid, verbose = 3, n_jobs = -1)
grid1.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:   25.7s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  4.8min finished
/Users/jiaping/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000],
                         'max_iter': [1000, 5000, 10000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'lbfgs']},
             verbose=3)

In [25]:
print(f"The best paramaters for the tuned model: {grid1.best_params_}\n") 
print(f"The cross-validated accuracy for the tuned model with normalized training set:{round(grid1.best_score_*100,2)}%")

The best paramaters for the tuned model: {'C': 1000, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'lbfgs'}

The cross-validated accuracy for the tuned model with normalized training set:88.9%


After model tuning, the cross-validated accuracy is increased from 84.89% to 88.9%.

Results on the whole training set

In [26]:
predictions1 = grid1.predict(X_train_scaled)
print(classification_report(y_train, predictions1))
print(f"{round(grid1.score(X_train_scaled, y_train)*100,2)}%")  # 88.44%, 89.3%

                precision    recall  f1-score   support

     CANDIDATE       0.84      0.70      0.76      1586
     CONFIRMED       0.76      0.86      0.81      1704
FALSE POSITIVE       0.98      1.00      0.99      3268

      accuracy                           0.89      6558
     macro avg       0.86      0.85      0.85      6558
  weighted avg       0.89      0.89      0.89      6558

88.9%


### 2. Standardized data

In [27]:
grid2 = GridSearchCV(model, param_grid2, verbose = 3, n_jobs = -1)
grid2.fit(X_train_stand, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:   33.1s finished


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000],
                         'max_iter': [1000, 5000, 10000]},
             verbose=3)

In [28]:
print(f"The best paramaters for the tuned model: {grid2.best_params_}\n")
print(f"The cross-validated accuracy for the tuned model with standardized training set:{round(grid2.best_score_*100,2)}%")

The best paramaters for the tuned model: {'C': 1000, 'max_iter': 5000}

The cross-validated accuracy for the tuned model with standardized training set:89.04%


After model tuning, the cross-validated accuracy is increased from 88.67% to 89.04%.

Results on the whole training set

In [29]:
predictions2 = grid2.predict(X_train_stand)
print(classification_report(y_train, predictions2))
print(f"{round(grid2.score(X_train_stand, y_train)*100,2)}%")

                precision    recall  f1-score   support

     CANDIDATE       0.84      0.71      0.77      1586
     CONFIRMED       0.77      0.86      0.82      1704
FALSE POSITIVE       0.98      1.00      0.99      3268

      accuracy                           0.89      6558
     macro avg       0.87      0.86      0.86      6558
  weighted avg       0.89      0.89      0.89      6558

89.34%


The tuned model with standardized data perfoems slightly better than the tuned model with normalized data, as the cross-validated accuracy of the tuned model with standardized data (89.04%) is higher than the one with normalized data (88.9%).

The final logistic regression model :
1. Requires the standardization of data.
2. Model parameters:  {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
3. Cross-validated accuracy: 89.04%

# Save the tuned model

In [33]:
import joblib
filename = 'logistic_regression.sav'
joblib.dump(grid2, filename)

['logistic_regression.sav']